In [2]:
%%bash
eu_register.py --help

usage: eu_register.py [-h] [-m DCC_MODE] [-d] [--no-aliases] -p PROFILE_ID -i
                      INFILE [--patch] [-w]

Given a tab-delimited or JSON input file containing one or more records belonging to one of the profiles
listed on the ENCODE Portal (such as https://www.encodeproject.org/profiles/biosample.json),
either POSTS or PATCHES the records. The default is to POST each record; to PATCH instead, see
the ``--patch`` option.

When POSTING file records, the md5sum of each file will be calculated for you if you haven't
already provided the `md5sum` property. Then, after the POST operation completes, the actual file
will be uploaded to AWS S3. In order for this to work, you must set the `submitted_file_name`
property to the full, local path to your file to upload. Alternatively, you can set
`submitted_file_name` to and existing S3 object, i.e. s3://mybucket/reads.fastq.

Note that there is a special 'trick' defined in the ``encode_utils.connection.Connection()``
class that can 

In [3]:
!ls /data/reddylab/Revathy/collabs/Keith/ENCODE_submission

Reddy-DCC-Siklenka-mmCD4-PooledASTARRData.xlsx	Untitled.ipynb


In [5]:
%%bash
mkdir -p /data/reddylab/Revathy/collabs/Keith/ENCODE_submission/metadata
source /data/reddylab/software/miniconda3/bin/activate revathy_py3
python /data/reddylab/Revathy/scripts/excel_to_text_for_ENCODE.py \
-i /data/reddylab/Revathy/collabs/Keith/ENCODE_submission/Reddy-DCC-Siklenka-mmCD4-PooledASTARRData.xlsx \
-o /data/reddylab/Revathy/collabs/Keith/ENCODE_submission/metadata \
--sheet-names genetic_modification biosample library functional_characterization_exp replicate

In [1]:
%%bash
source /data/reddylab/software/miniconda3/bin/activate revathy_py3
cd /data/reddylab/Revathy/collabs/Keith/ENCODE_submission/metadata
sbatch -p all \
    --mem 16G \
    <<'EOF'
#!/bin/bash
NAMES=( \
     biosample_v1 \
)
NAME=${NAMES[${SLURM_ARRAY_TASK_ID}]}
eu_register.py -m prod -p file -i file.txt \
EOF

Submitted batch job 25924734


In [1]:
%%bash
source /data/reddylab/software/miniconda3/bin/activate revathy_py3
python /data/reddylab/Revathy/scripts/excel_to_text_for_ENCODE.py \
-i /data/reddylab/Revathy/collabs/Keith/ENCODE_submission/Reddy-DCC-Siklenka-mmCD4-PooledASTARRData-files.xlsx \
-o /data/reddylab/Revathy/collabs/Keith/ENCODE_submission/metadata \
--sheet-names fastq bam bigWigs

In [7]:
%%bash
source /data/reddylab/software/miniconda3/bin/activate revathy_py3
cd /data/reddylab/Revathy/collabs/Keith/ENCODE_submission/metadata
sbatch -p all \
    -o /data/reddylab/Revathy/collabs/Keith/ENCODE_submission/logs/file.out \
    --mem 16G \
    <<'EOF'
#!/bin/bash
NAMES=( \
    bigWigs \
)
NAME=${NAMES[${SLURM_ARRAY_TASK_ID}]}
eu_register.py -m prod -p file -i ${NAME}.txt \
EOF

Submitted batch job 25913060


In [5]:
%%bash
eu_register.py -m prod -p file -i /data/reddylab/Revathy/collabs/Keith/ENCODE_submission/metadata/bam.txt

Process is terminated.


In [2]:
%%bash
eu_register.py -m prod -p file -i /data/reddylab/Revathy/collabs/Keith/ENCODE_submission/metadata/bam-v1.txt

2021-11-01 15:27:49,257:eu_debug:	Connecting to www.encodeproject.org
2021-11-01 15:27:49,269:eu_debug:	submission=False: In non-submission mode.
2021-11-01 15:27:50,034:eu_debug:	submission=True: In submission mode.
2021-11-01 15:27:50,039:eu_debug:	
IN post().
2021-11-01 15:27:56,854:eu_debug:	<<<<<< POST file record tim-reddy:mmCD4-atac-starrseq-input-rep4-bam To DCC with URL https://www.encodeproject.org/file and this payload:

{
  "aliases": [
    "tim-reddy:mmCD4-atac-starrseq-input-rep4-bam"
  ],
  "assembly": "mm10",
  "award": "/awards/UM1HG009428/",
  "dataset": "tim-reddy:mmCD4-atac-starrseq-input",
  "derived_from": [
    "/files/mm10_no_alt_analysis_set_ENCODE/",
    "tim-reddy:mmCD4-atac-starrseq-input-rep4-read1",
    "tim-reddy:mmCD4-atac-starrseq-input-rep4-read2"
  ],
  "file_format": "bam",
  "file_size": 3486909877,
  "lab": "/labs/tim-reddy/",
  "md5sum": "40f188d49f854494681cecb17226733b",
  "output_type": "alignments",
  "submitted_file_name": "/data/reddylab/Kei

In [2]:
import encode_utils as eu
from encode_utils.connection import Connection


conn = Connection("prod")

2022-08-15 15:56:09,953:eu_debug:	Connecting to www.encodeproject.org
2022-08-15 15:56:09,989:eu_debug:	submission=False: In non-submission mode.


In [7]:
import pandas as pd

corrected_fastq = pd.read_csv('/data/reddylab/Revathy/collabs/Keith/ENCODE_submission/metadata/corrected_fastq_v1.txt', sep='\t')
corrected_fastq

,file_id,file_path
0,ENCFF901YSP,/data/reddylab/Keith/encode4_duke/data/starr_s...
1,ENCFF827YMY,/data/reddylab/Keith/encode4_duke/data/starr_s...


In [8]:
for n in range(0,len(corrected_fastq)):
    conn.upload_file(file_id=corrected_fastq.file_id[n], file_path=corrected_fastq.file_path[n])

2021-11-01 15:34:54,981:eu_debug:	
IN upload_file()

2021-11-01 15:34:54,983:eu_debug:	Attempting to generate new file upload credentials
2021-11-01 15:34:55,746:eu_debug:	Success: upload credentials for 'ENCFF901YSP' regenerated.
2021-11-01 15:34:55,747:eu_debug:	>>>>>>GET ENCFF901YSP From DCC with URL https://www.encodeproject.org/ENCFF901YSP/?format=json
2021-11-01 15:34:56,335:eu_debug:	Running command 'aws s3 cp /data/reddylab/Keith/encode4_duke/data/starr_seq/processed_raw_reads/210528_KS163_ASTARRPooled_Output_NextSeq/KS163_Th17_pooledASTARR_output_rep4.R1.fastq.gz s3://encode-files/2021/08/31/af430bc7-b15c-4316-bc93-19cccbcb1079/ENCFF901YSP.fastq.gz'.
2021-11-01 15:35:20,258:eu_debug:	AWS upload successful.
2021-11-01 15:35:20,262:eu_debug:	
IN upload_file()

2021-11-01 15:35:20,263:eu_debug:	Attempting to generate new file upload credentials
2021-11-01 15:35:21,032:eu_debug:	Success: upload credentials for 'ENCFF827YMY' regenerated.
2021-11-01 15:35:21,033:eu_debug:	>>>>>>GET 

In [9]:
conn.upload_file(file_id='ENCFF538JII', file_path='/data/reddylab/Revathy/collabs/Keith/ENCODE_submission/metadata/bam-v1.txt')

2021-11-01 15:36:34,763:eu_debug:	
IN upload_file()

2021-11-01 15:36:34,764:eu_debug:	Attempting to generate new file upload credentials
2021-11-01 15:36:35,575:eu_debug:	Success: upload credentials for 'ENCFF538JII' regenerated.
2021-11-01 15:36:35,576:eu_debug:	>>>>>>GET ENCFF538JII From DCC with URL https://www.encodeproject.org/ENCFF538JII/?format=json
2021-11-01 15:36:36,167:eu_debug:	Running command 'aws s3 cp /data/reddylab/Revathy/collabs/Keith/ENCODE_submission/metadata/bam-v1.txt s3://encode-files/2021/08/31/e36776f9-ff95-415c-acac-cc33d0aa54ad/ENCFF538JII.bam'.
2021-11-01 15:36:40,387:eu_debug:	AWS upload successful.


In [11]:
import pandas as pd

corrected_bam_bigWig = pd.read_csv('/data/reddylab/Revathy/collabs/Keith/ENCODE_submission/metadata/allPooled_atacstarr_patch_bam_bigWig.txt', sep='\t')
corrected_bam_bigWig.head()

,file_id,file_path
0,ENCFF023WVD,/data/reddylab/Revathy/collabs/Keith/ENCODE_su...
1,ENCFF151HHA,/data/reddylab/Revathy/collabs/Keith/ENCODE_su...
2,ENCFF731CKD,/data/reddylab/Revathy/collabs/Keith/ENCODE_su...
3,ENCFF805BBU,/data/reddylab/Revathy/collabs/Keith/ENCODE_su...
4,ENCFF382RRF,/data/reddylab/Revathy/collabs/Keith/ENCODE_su...


In [12]:
for n in range(0,len(corrected_bam_bigWig)):
    conn.upload_file(file_id=corrected_bam_bigWig.file_id[n], file_path=corrected_bam_bigWig.file_path[n])

2021-09-13 14:14:17,337:eu_debug:	
IN upload_file()

2021-09-13 14:14:17,339:eu_debug:	Attempting to generate new file upload credentials
2021-09-13 14:14:18,185:eu_debug:	Success: upload credentials for 'ENCFF023WVD' regenerated.
2021-09-13 14:14:18,187:eu_debug:	>>>>>>GET ENCFF023WVD From DCC with URL https://www.encodeproject.org/ENCFF023WVD/?format=json
2021-09-13 14:14:18,764:eu_debug:	Running command 'aws s3 cp /data/reddylab/Revathy/collabs/Keith/ENCODE_submission/data/KS151_AllCD4Pooled_ASTARR_Input_rep1.masked.dups_marked.encode.bam s3://encode-files/2021/08/31/696aafb4-bbc9-430b-9cd4-fac6971083ed/ENCFF023WVD.bam'.
2021-09-13 14:14:53,856:eu_debug:	AWS upload successful.
2021-09-13 14:14:53,859:eu_debug:	
IN upload_file()

2021-09-13 14:14:53,861:eu_debug:	Attempting to generate new file upload credentials
2021-09-13 14:14:54,600:eu_debug:	Success: upload credentials for 'ENCFF151HHA' regenerated.
2021-09-13 14:14:54,601:eu_debug:	>>>>>>GET ENCFF151HHA From DCC with URL https:

In [1]:
import pandas as pd

corrected_fastq_v1 = pd.read_csv('/data/reddylab/Revathy/collabs/Keith/ENCODE_submission/metadata/corrected_fastq_v1.txt', sep='\t')
corrected_fastq_v1.file_path[1]

'/data/reddylab/Keith/encode4_duke/data/starr_seq/processed_raw_reads/210528_KS163_ASTARRPooled_Output_NextSeq/KS163_Th17_Th17ASTARR_output_rep4.R2.fastq.gz'

In [3]:
for n in range(0,len(corrected_fastq_v1)):
    conn.upload_file(file_id=corrected_fastq_v1.file_id[n], file_path=corrected_fastq_v1.file_path[n])

2021-09-15 12:05:40,318:eu_debug:	
IN upload_file()

2021-09-15 12:05:40,320:eu_debug:	Attempting to generate new file upload credentials
2021-09-15 12:05:41,118:eu_debug:	Success: upload credentials for 'ENCFF681ALK' regenerated.
2021-09-15 12:05:41,120:eu_debug:	>>>>>>GET ENCFF681ALK From DCC with URL https://www.encodeproject.org/ENCFF681ALK/?format=json
2021-09-15 12:05:41,716:eu_debug:	Running command 'aws s3 cp /data/reddylab/Keith/encode4_duke/data/starr_seq/processed_raw_reads/210528_KS163_ASTARRPooled_Output_NextSeq/KS163_Th17_Th17ASTARR_output_rep4.R1.fastq.gz s3://encode-files/2021/09/10/7e0af70b-000f-4b48-a38a-dc08134cd023/ENCFF681ALK.fastq.gz'.
2021-09-15 12:05:57,704:eu_debug:	AWS upload successful.
2021-09-15 12:05:57,707:eu_debug:	
IN upload_file()

2021-09-15 12:05:57,709:eu_debug:	Attempting to generate new file upload credentials
2021-09-15 12:05:58,475:eu_debug:	Success: upload credentials for 'ENCFF043LTV' regenerated.
2021-09-15 12:05:58,477:eu_debug:	>>>>>>GET EN

In [4]:
conn.upload_file(file_id='ENCFF464BLK', file_path='/data/reddylab//Revathy/collabs/Keith/ENCODE_submission/data/KS180_Th2_pooledASTARR_output_rep1.f3q10.sorted.with_umis.dups_marked.encode.bam')

2022-08-15 15:58:49,789:eu_debug:	
IN upload_file()

2022-08-15 15:58:49,790:eu_debug:	Attempting to generate new file upload credentials
2022-08-15 15:58:50,591:eu_debug:	Success: upload credentials for 'ENCFF464BLK' regenerated.
2022-08-15 15:58:50,594:eu_debug:	>>>>>>GET ENCFF464BLK From DCC with URL https://www.encodeproject.org/ENCFF464BLK/?format=json
2022-08-15 15:58:51,273:eu_debug:	Running command 'aws s3 cp /data/reddylab//Revathy/collabs/Keith/ENCODE_submission/data/KS180_Th2_pooledASTARR_output_rep1.f3q10.sorted.with_umis.dups_marked.encode.bam s3://encode-files/2021/08/31/a7541a37-b5af-4569-ac82-8ccc9e5aa225/ENCFF464BLK.bam'.
2022-08-15 15:59:14,229:eu_debug:	AWS upload successful.


In [1]:
%%bash
source /data/reddylab/software/miniconda3/bin/activate revathy_py3
cd /data/reddylab/Revathy/collabs/Keith/ENCODE_submission/metadata
eu_register.py -m prod -p file -i patch_md5sum.txt --patch -w

2021-09-13 13:28:20,577:eu_debug:	Connecting to www.encodeproject.org
2021-09-13 13:28:20,599:eu_debug:	submission=False: In non-submission mode.
2021-09-13 13:28:21,327:eu_debug:	submission=True: In submission mode.
2021-09-13 13:28:21,355:eu_debug:	
IN patch()
2021-09-13 13:28:21,355:eu_debug:	>>>>>>GET ENCFF900BNA From DCC with URL https://www.encodeproject.org/ENCFF900BNA/?format=json&datastore=database&frame=edit
2021-09-13 13:28:21,707:eu_debug:	<<<<<< PATCHING ENCFF900BNA To DCC with URL https://www.encodeproject.org/ENCFF900BNA and this payload:

{
  "md5sum": "2c829f7a5058e61d8a7c3bd0a81dd355"
}


2021-09-13 13:28:22,132:eu_debug:	Success.
2021-09-13 13:28:22,132:eu_debug:	
IN patch()
2021-09-13 13:28:22,133:eu_debug:	>>>>>>GET ENCFF827YMY From DCC with URL https://www.encodeproject.org/ENCFF827YMY/?format=json&datastore=database&frame=edit
2021-09-13 13:28:22,486:eu_debug:	<<<<<< PATCHING ENCFF827YMY To DCC with URL https://www.encodeproject.org/ENCFF827YMY and this payload:


In [15]:
!cat /data/reddylab/Revathy/collabs/Keith/ENCODE_submission/metadata/patch_md5sum.txt







































ENCFF144PBX	e9fce54be8a2dfcb5037bfe642b6c54d

In [1]:
%%bash
source /data/reddylab/software/miniconda3/bin/activate revathy_py3
cd /data/reddylab/Revathy/collabs/Keith/ENCODE_submission/metadata
eu_register.py -m prod -p file -i patch_md5sum-v1.txt --patch -w

2021-09-29 14:42:35,608:eu_debug:	Connecting to www.encodeproject.org
2021-09-29 14:42:35,644:eu_debug:	submission=False: In non-submission mode.
2021-09-29 14:42:36,997:eu_debug:	submission=True: In submission mode.
2021-09-29 14:42:37,004:eu_debug:	
IN patch()
2021-09-29 14:42:37,004:eu_debug:	>>>>>>GET ENCFF681ALK From DCC with URL https://www.encodeproject.org/ENCFF681ALK/?format=json&datastore=database&frame=edit
2021-09-29 14:42:37,344:eu_debug:	<<<<<< PATCHING ENCFF681ALK To DCC with URL https://www.encodeproject.org/ENCFF681ALK and this payload:

{
  "md5sum": "71dd30ff14ec164e1c5d8158e167a6c6"
}


2021-09-29 14:42:37,821:eu_debug:	Success.
2021-09-29 14:42:37,822:eu_debug:	
IN patch()
2021-09-29 14:42:37,822:eu_debug:	>>>>>>GET ENCFF043LTV From DCC with URL https://www.encodeproject.org/ENCFF043LTV/?format=json&datastore=database&frame=edit
2021-09-29 14:42:38,149:eu_debug:	<<<<<< PATCHING ENCFF043LTV To DCC with URL https://www.encodeproject.org/ENCFF043LTV and this payload:


### Patch bam and bigWig analysis step run 

In [5]:
%%bash
source /data/reddylab/software/miniconda3/bin/activate revathy_py3
cd /data/reddylab/Revathy/collabs/Keith/ENCODE_submission/metadata
eu_register.py -m prod -p file -i bam_step_run_v1.txt --patch -w

2022-09-14 14:38:06,857:eu_debug:	Connecting to www.encodeproject.org
2022-09-14 14:38:06,858:eu_debug:	submission=False: In non-submission mode.
2022-09-14 14:38:07,580:eu_debug:	submission=True: In submission mode.
2022-09-14 14:38:07,606:eu_debug:	
IN patch()
2022-09-14 14:38:07,606:eu_debug:	>>>>>>GET files/ENCFF023WVD From DCC with URL https://www.encodeproject.org/files/ENCFF023WVD/?format=json&datastore=database&frame=edit
2022-09-14 14:38:07,984:eu_debug:	<<<<<< PATCHING /files/ENCFF023WVD/ To DCC with URL https://www.encodeproject.org/files/ENCFF023WVD and this payload:

{
  "step_run": "tim-reddy:starrseq_fastq_to_bam_no_umi_step_run"
}


2022-09-14 14:38:08,438:eu_debug:	Success.
2022-09-14 14:38:08,438:eu_debug:	
IN patch()
2022-09-14 14:38:08,438:eu_debug:	>>>>>>GET files/ENCFF151HHA From DCC with URL https://www.encodeproject.org/files/ENCFF151HHA/?format=json&datastore=database&frame=edit
2022-09-14 14:38:08,791:eu_debug:	<<<<<< PATCHING /files/ENCFF151HHA/ To DCC with U

In [4]:
%%bash
source /data/reddylab/software/miniconda3/bin/activate revathy_py3
cd /data/reddylab/Revathy/collabs/Keith/ENCODE_submission/metadata
eu_register.py -m prod -p file -i bigwig_step_run_v1.txt --patch -w

2022-09-14 14:37:20,175:eu_debug:	Connecting to www.encodeproject.org
2022-09-14 14:37:20,176:eu_debug:	submission=False: In non-submission mode.
2022-09-14 14:37:20,872:eu_debug:	submission=True: In submission mode.
2022-09-14 14:37:20,878:eu_debug:	
IN patch()
2022-09-14 14:37:20,878:eu_debug:	>>>>>>GET files/ENCFF485YLA From DCC with URL https://www.encodeproject.org/files/ENCFF485YLA/?format=json&datastore=database&frame=edit
2022-09-14 14:37:21,247:eu_debug:	<<<<<< PATCHING /files/ENCFF485YLA/ To DCC with URL https://www.encodeproject.org/files/ENCFF485YLA and this payload:

{
  "step_run": "tim-reddy:starrseq_bam_to_bigwig_deeptools_step_run"
}


2022-09-14 14:37:21,783:eu_debug:	Success.
2022-09-14 14:37:21,784:eu_debug:	
IN patch()
2022-09-14 14:37:21,784:eu_debug:	>>>>>>GET files/ENCFF374WNU From DCC with URL https://www.encodeproject.org/files/ENCFF374WNU/?format=json&datastore=database&frame=edit
2022-09-14 14:37:22,156:eu_debug:	<<<<<< PATCHING /files/ENCFF374WNU/ To DCC wi

In [5]:
!cat /data/reddylab/Revathy/collabs/Keith/ENCODE_submission/metadata/bigwig_step_run_v1.txt

record_id	step_run
/files/ENCFF485YLA/	tim-reddy:starrseq_bam_to_bigwig_deeptools_step_run
/files/ENCFF374WNU/	tim-reddy:starrseq_bam_to_bigwig_deeptools_step_run
/files/ENCFF457WDD/	tim-reddy:starrseq_bam_to_bigwig_deeptools_step_run
/files/ENCFF360KNY/	tim-reddy:starrseq_bam_to_bigwig_deeptools_step_run
/files/ENCFF719ARU/	tim-reddy:starrseq_bam_to_bigwig_deeptools_step_run
/files/ENCFF640EZO/	tim-reddy:starrseq_bam_to_bigwig_deeptools_step_run

### Check md5sum for bams with content error

In [ ]:
%%bash
cd /data/reddylab/Revathy/collabs/Keith/ENCODE_submission/metadata/tmp/
curl -SLo mmTh17.rep1.ENCFF740KDN.bam \
    https://www.encodeproject.org/files/ENCFF331URE/@@download/ENCFF331URE.bigWig

In [11]:
%%bash
cd /data/reddylab/Revathy/collabs/Keith/ENCODE_submission/metadata/tmp/
md5sum mmTh17.rep1.ENCFF740KDN.bam
md5sum mmTh17.rep2.ENCFF269RKV.bam
md5sum mmTh2.rep1.ENCFF464BLK.bam
md5sum mmTh2.rep2.ENCFF586ZAW.bam


e6734d5cbf930dda8a8ff3d3e4c8b072  mmTh17.rep1.ENCFF740KDN.bam
7db9d046812b479f2be3505647478d98  mmTh17.rep2.ENCFF269RKV.bam
b75df9891c23af38bb74a0d076fe07ce  mmTh2.rep1.ENCFF464BLK.bam
1eba3cdbe1591f420b98d93602c3e3c4  mmTh2.rep2.ENCFF586ZAW.bam


In [3]:
%%bash
cd /data/reddylab/Revathy/collabs/Keith/ENCODE_submission/data/
md5sum KS187_Th2_ASTARR_allPool_output_Rep2.f3q10.sorted.with_umis.dups_marked.encode.bam
md5sum KS180_Th2_pooledASTARR_output_rep1.f3q10.sorted.with_umis.dups_marked.encode.bam
md5sum KS151_Th17_ASTARR_pool_rep1.f3q10.sorted.with_umis.dups_marked.encode.bam
md5sum KS151_Th17_ASTARR_pool_rep2.f3q10.sorted.with_umis.dups_marked.encode.bam

2a2d2debf311f7287e3a5d69589001eb  KS187_Th2_ASTARR_allPool_output_Rep2.f3q10.sorted.with_umis.dups_marked.encode.bam
ea7196c61e662a0a514c74cc3ed60f35  KS180_Th2_pooledASTARR_output_rep1.f3q10.sorted.with_umis.dups_marked.encode.bam
79186928f6d490eb555648761662536c  KS151_Th17_ASTARR_pool_rep1.f3q10.sorted.with_umis.dups_marked.encode.bam
1b82c43f658e5e74615a24483a7f2d90  KS151_Th17_ASTARR_pool_rep2.f3q10.sorted.with_umis.dups_marked.encode.bam


In [13]:
%%bash
cd /data/reddylab/Revathy/collabs/Keith/ENCODE_submission/data/Th17
md5sum KS148_Th17_ASTARR_Th17Inp_rep1.f3q10.sorted.with_umis.dups_marked.encode.bam
md5sum KS149_Th17_ASTARR_Th17Inp_rep2.f3q10.sorted.with_umis.dups_marked.encode.bam

e45cce2094fbcaaf0b1a907bd5e828c5  KS148_Th17_ASTARR_Th17Inp_rep1.f3q10.sorted.with_umis.dups_marked.encode.bam
c401a46d365177b148562b886b47008f  KS149_Th17_ASTARR_Th17Inp_rep2.f3q10.sorted.with_umis.dups_marked.encode.bam


436fd2f0eb09ffec89156c051c575cbf - /data/reddylab/Keith/encode4_duke/processing/starr_seq/210528_KS163_ASTARRPooled_Output_NextSeq-pe-umis/KS180_Th2_pooledASTARR_output_rep1.f3q10.sorted.with_umis.dups_marked.bam
240b26b0cdeef2b5ec65b2f6256c7205 - /data/reddylab/Keith/encode4_duke/processing/starr_seq/210731_KS183_Th1_PooledASTARR_Output_NextSeq-pe-umis/KS187_Th2_ASTARR_allPool_output_Rep2.f3q10.sorted.with_umis.dups_marked.bam
39d3241816cd18159c7a76206c9a96cc - /data/reddylab/Keith/encode4_duke/processing/starr_seq/201109_KS151_Th17ASTARR_pool2-pe-umis/KS151_Th17_ASTARR_pool_rep1.f3q10.sorted.with_umis.dups_marked.bam
e17b80feba63590b1e5475fac19ac509 - /data/reddylab/Keith/encode4_duke/processing/starr_seq/201109_KS151_Th17ASTARR_pool2-pe-umis/KS151_Th17_ASTARR_pool_rep2.f3q10.sorted.with_umis.dups_marked.bam

In [1]:
import encode_utils as eu
from encode_utils.connection import Connection


conn = Connection("prod")

conn.upload_file(file_id='ENCFF740KDN', file_path='/data/reddylab/Revathy/collabs/Keith/ENCODE_submission/data/KS151_Th17_ASTARR_pool_rep1.f3q10.sorted.with_umis.dups_marked.encode.bam')
conn.upload_file(file_id='ENCFF269RKV', file_path='/data/reddylab/Revathy/collabs/Keith/ENCODE_submission/data/KS151_Th17_ASTARR_pool_rep2.f3q10.sorted.with_umis.dups_marked.encode.bam')


2022-09-15 12:51:42,380:eu_debug:	Connecting to www.encodeproject.org
2022-09-15 12:51:42,416:eu_debug:	submission=False: In non-submission mode.
2022-09-15 12:51:43,169:eu_debug:	
IN upload_file()

2022-09-15 12:51:43,170:eu_debug:	Attempting to generate new file upload credentials
2022-09-15 12:51:44,014:eu_debug:	Success: upload credentials for 'ENCFF740KDN' regenerated.
2022-09-15 12:51:44,017:eu_debug:	>>>>>>GET ENCFF740KDN From DCC with URL https://www.encodeproject.org/ENCFF740KDN/?format=json
2022-09-15 12:51:44,742:eu_debug:	Running command 'aws s3 cp /data/reddylab/Revathy/collabs/Keith/ENCODE_submission/data/KS151_Th17_ASTARR_pool_rep1.f3q10.sorted.with_umis.dups_marked.encode.bam s3://encode-files/2021/08/31/1792df89-21e5-4ae9-85e3-fda2d0d966db/ENCFF740KDN.bam'.
2022-09-15 12:52:28,915:eu_debug:	AWS upload successful.
2022-09-15 12:52:28,919:eu_debug:	
IN upload_file()

2022-09-15 12:52:28,920:eu_debug:	Attempting to generate new file upload credentials
2022-09-15 12:52:29,

In [2]:
%%bash
md5sum /data/reddylab/Revathy/collabs/Keith/ENCODE_submission/data/KS151_Th17_ASTARR_pool_rep1.f3q10.sorted.with_umis.dups_marked.encode.bam
md5sum /data/reddylab/Revathy/collabs/Keith/ENCODE_submission/data/KS151_Th17_ASTARR_pool_rep2.f3q10.sorted.with_umis.dups_marked.encode.bam

79186928f6d490eb555648761662536c  /data/reddylab/Revathy/collabs/Keith/ENCODE_submission/data/KS151_Th17_ASTARR_pool_rep1.f3q10.sorted.with_umis.dups_marked.encode.bam
1b82c43f658e5e74615a24483a7f2d90  /data/reddylab/Revathy/collabs/Keith/ENCODE_submission/data/KS151_Th17_ASTARR_pool_rep2.f3q10.sorted.with_umis.dups_marked.encode.bam


In [1]:
%%bash
source /data/reddylab/software/miniconda3/bin/activate revathy_py3
cd /data/reddylab/Revathy/collabs/Keith/ENCODE_submission/metadata
eu_register.py -m prod -p reference -i reference_object.txt 

2022-10-24 14:33:36,209:eu_debug:	Connecting to www.encodeproject.org
2022-10-24 14:33:36,255:eu_debug:	submission=False: In non-submission mode.
2022-10-24 14:33:37,097:eu_debug:	submission=True: In submission mode.
2022-10-24 14:33:37,102:eu_debug:	
IN post().
2022-10-24 14:33:37,102:eu_debug:	<<<<<< POST reference record tim-reddy:mmCD4-atac-starrseq-input-reference To DCC with URL https://www.encodeproject.org/reference and this payload:

{
  "aliases": [
    "tim-reddy:mmCD4-atac-starrseq-input-reference"
  ],
  "award": "/awards/UM1HG009428/",
  "elements_selection_method": [
    "accessible genome regions"
  ],
  "lab": "/labs/tim-reddy/",
  "reference_type": "functional elements"
}


2022-10-24 14:33:37,503:eu_debug:	Success.


### Rep2 biosample for all cell types

In [4]:
%%bash
source /data/reddylab/software/miniconda3/bin/activate revathy_py3
cd /data/reddylab/Revathy/collabs/Keith/ENCODE_submission/metadata
eu_register.py -m prod -p biosample -i biosample_v2.txt

2023-01-23 13:31:49,943:eu_debug:	Connecting to www.encodeproject.org
2023-01-23 13:31:49,944:eu_debug:	submission=False: In non-submission mode.
2023-01-23 13:31:50,695:eu_debug:	submission=True: In submission mode.
2023-01-23 13:31:50,700:eu_debug:	
IN post().
2023-01-23 13:31:50,701:eu_debug:	<<<<<< POST biosample record tim-reddy:mmTh0-atac-starrseq-biosample-rep2 To DCC with URL https://www.encodeproject.org/biosample and this payload:

{
  "aliases": [
    "tim-reddy:mmTh0-atac-starrseq-biosample-rep2"
  ],
  "award": "/awards/UM1HG009428/",
  "biosample_ontology": "/biosample-types/primary_cell_CL_0000492/",
  "donor": "encode:C57BL_6NJ",
  "genetic_modifications": [
    "tim-reddy:mmCD4-atac-starrseq-input-genetic-modification"
  ],
  "lab": "/labs/tim-reddy/",
  "organism": "mouse",
  "source": "/sources/maria-ciofani/"
}


2023-01-23 13:31:51,310:eu_debug:	{'status': 'error', 'description': 'There was a conflict when trying to complete your request.', 'detail': "Keys conflict

In [4]:
%%bash
source /data/reddylab/software/miniconda3/bin/activate revathy_py3
cd /data/reddylab/Revathy/collabs/Keith/ENCODE_submission/metadata
eu_register.py -m prod -p library -i library_v1.txt --patch -w

2023-02-27 12:50:49,209:eu_debug:	Connecting to www.encodeproject.org
2023-02-27 12:50:49,210:eu_debug:	submission=False: In non-submission mode.
2023-02-27 12:50:49,915:eu_debug:	submission=True: In submission mode.


Traceback (most recent call last):
  File "/data/reddylab/software/miniconda3/envs/revathy_py3/bin/eu_register.py", line 345, in <module>
    main()
  File "/data/reddylab/software/miniconda3/envs/revathy_py3/bin/eu_register.py", line 162, in main
    euu.print_format_dict(payload), RECORD_ID_FIELD))
Exception: Can't patch payload {
  "_profile": "library",
  "aliases": [
    "tim-reddy:mmTh0-atac-starrseq-rep2-lib"
  ],
  "award": "/awards/UM1HG009428/",
  "biosample": "tim-reddy:mmTh0-atac-starrseq-biosample-rep2",
  "extraction_method": "Qiagen RNA extraction",
  "lab": "/labs/tim-reddy/",
  "nucleic_acid_starting_quantity": 30.0,
  "nucleic_acid_starting_quantity_units": "\u00b5g",
  "nucleic_acid_term_name": "RNA",
  "size_range": "200-400",
  "strand_specificity": "unstranded"
} since there isn't a 'record_id' field indiciating an identifer for the record to be PATCHED.


### Upload the CSAW peak calls for the all-pooled CD4+ T cells

/home/ks476/data_keith/encode4_duke/analysis/220118_KS151_AllPooled_ASTARR_Analysis/221010_csaw_iterations/v9/221017.atacSTARR.pooled.Th0.3window_minTest.weighted.csaw.mm10.noATACpeaks.0bp_min.0.01.v9.tsv
/home/ks476/data_keith/encode4_duke/analysis/220118_KS151_AllPooled_ASTARR_Analysis/221010_csaw_iterations/v9/221017.atacSTARR.pooled.Th1.3window_minTest.weighted.csaw.mm10.noATACpeaks.0bp_min.0.01.v9.tsv
/home/ks476/data_keith/encode4_duke/analysis/220118_KS151_AllPooled_ASTARR_Analysis/221010_csaw_iterations/v9/221017.atacSTARR.pooled.Th17.3window_minTest.weighted.csaw.mm10.noATACpeaks.0bp_min.0.01.v9.tsv
/home/ks476/data_keith/encode4_duke/analysis/220118_KS151_AllPooled_ASTARR_Analysis/221010_csaw_iterations/v9/221017.atacSTARR.pooled.Th2.3window_minTest.weighted.csaw.mm10.noATACpeaks.0bp_min.0.01.v9.tsv
/home/ks476/data_keith/encode4_duke/analysis/220118_KS151_AllPooled_ASTARR_Analysis/221010_csaw_iterations/v9/221017.atacSTARR.pooled.Treg.3window_minTest.weighted.csaw.mm10.noATACpeaks.0bp_min.0.01.v9.tsv

In [12]:
!head /home/ks476/data_keith/encode4_duke/analysis/220118_KS151_AllPooled_ASTARR_Analysis/221010_csaw_iterations/v9/221017.atacSTARR.pooled.Treg.3window_minTest.weighted.csaw.mm10.noATACpeaks.0bp_min.0.01.v9.tsv

seqnames	start	end	width	strand	num.tests	num.up.logFC	num.down.logFC	PValue	FDR	direction	rep.test	rep.logFC
chr1	4785276	4786100	825	*	32	0	13	0.000748899892826528	0.00619121270394489	down	189	-0.64758133548357
chr1	4807476	4808225	750	*	29	0	27	2.10728465771275e-05	0.000347794433178135	down	234	-1.28002815591567
chr1	4873426	4873750	325	*	12	8	0	6.59598880144626e-06	0.000136343248158472	up	289	2.40439360383632
chr1	4915951	4916275	325	*	12	11	0	1.15320581716254e-07	5.62763141301542e-06	up	320	2.08758526688802
chr1	5082701	5083600	900	*	35	0	17	1.56533687663687e-05	0.000274459081707986	down	417	-1.41706199776324
chr1	6214201	6215475	1275	*	50	0	19	0.000162870849856608	0.00181865579494353	down	522	-1.06333310463278
chr1	6382601	6383225	625	*	24	0	9	3.33424150510366e-05	0.00050403327704375	down	616	-1.60268716330111
chr1	7036576	7036875	300	*	10	10	0	2.90538012772687e-06	7.08720006406723e-05	up	842	2.95531678434107
chr1	9545076	9545725	650	*	25	7	0	0.00018410092241651	0.002004253686993

In [13]:
%%bash
mkdir -p /data/reddylab/Revathy/collabs/Keith/ENCODE_submission/data/csaw_peaks
cell_types=(\ Th0 \
              Th1 \
              Th17 \
              Th2 \
              Treg)
for cell_type in ${cell_types[@]};
do 
cat /home/ks476/data_keith/encode4_duke/analysis/220118_KS151_AllPooled_ASTARR_Analysis/221010_csaw_iterations/v9/221017.atacSTARR.pooled.${cell_type}.3window_minTest.weighted.csaw.mm10.noATACpeaks.0bp_min.0.01.v9.tsv \
| tail -n+2 \
| awk -vOFS="\t" '{print $1, $2, $3, ".", '100', ".", $13, $9, $10, $4}' \
> /data/reddylab/Revathy/collabs/Keith/ENCODE_submission/data/csaw_peaks/${cell_type}.atacSTARR.pooled.3window_minTest.weighted.csaw.mm10.noATACpeaks.0bp_min.0.01.bed
done

In [2]:
%%bash
cell_types=(\ Th0 \
              Th1 \
              Th17 \
              Th2 \
              Treg)
for cell_type in ${cell_types[@]};
do 
gzip -c /data/reddylab/Revathy/collabs/Keith/ENCODE_submission/data/csaw_peaks/${cell_type}.atacSTARR.pooled.3window_minTest.weighted.csaw.mm10.noATACpeaks.0bp_min.0.01.bed \
> /data/reddylab/Revathy/collabs/Keith/ENCODE_submission/data/csaw_peaks/${cell_type}.atacSTARR.pooled.3window_minTest.weighted.csaw.mm10.noATACpeaks.0bp_min.0.01.bed.gz
done

In [4]:
!ls -la /data/reddylab/Revathy/collabs/Keith/ENCODE_submission/data/csaw_peaks/

total 11552
drwxrwsr-x+ 2 rv103 reddylab    8192 Apr 24 12:40 .
drwxrwsr-x+ 4 rv103 reddylab    8192 Apr 21 13:52 ..
-rw-rw-r--  1 rv103 reddylab 1000730 Apr 21 14:44 Th0.atacSTARR.pooled.3window_minTest.weighted.csaw.mm10.noATACpeaks.0bp_min.0.01.bed
-rw-rw-r--  1 rv103 reddylab  396174 Apr 24 12:40 Th0.atacSTARR.pooled.3window_minTest.weighted.csaw.mm10.noATACpeaks.0bp_min.0.01.bed.gz
-rw-rw-r--  1 rv103 reddylab 2303162 Apr 21 14:44 Th17.atacSTARR.pooled.3window_minTest.weighted.csaw.mm10.noATACpeaks.0bp_min.0.01.bed
-rw-rw-r--  1 rv103 reddylab  913851 Apr 24 12:40 Th17.atacSTARR.pooled.3window_minTest.weighted.csaw.mm10.noATACpeaks.0bp_min.0.01.bed.gz
-rw-rw-r--  1 rv103 reddylab 1426265 Apr 21 14:44 Th1.atacSTARR.pooled.3window_minTest.weighted.csaw.mm10.noATACpeaks.0bp_min.0.01.bed
-rw-rw-r--  1 rv103 reddylab  562790 Apr 24 12:40 Th1.atacSTARR.pooled.3window_minTest.weighted.csaw.mm10.noATACpeaks.0bp_min.0.01.bed.gz
-rw-rw-r--  1 rv103 reddylab 1747226 Apr 21 14:44 Th2.atacSTAR

In [5]:
!cat /data/reddylab/Revathy/collabs/Keith/ENCODE_submission/metadata/mmCD4-PooledASTARRData-peak-calls.txt 

dataset	file_format	file_format_type	output_type	submitted_file_name	award	lab	aliases	assembly	derived_from
tim-reddy:mmTh0-atac-starrseq	bed	narrowPeak	element quantifications	/data/reddylab/Revathy/collabs/Keith/ENCODE_submission/data/csaw_peaks/Th0.atacSTARR.pooled.3window_minTest.weighted.csaw.mm10.noATACpeaks.0bp_min.0.01.bed.gz	/awards/UM1HG009428/	/labs/tim-reddy/	tim-reddy:mmTh0-atac-starrseq-csaw-peakcall	mm10	tim-reddy:mmTh0-atac-starrseq-rep1-bam, tim-reddy:mmTh0-atac-starrseq-rep2-bam
tim-reddy:mmTh1-atac-starrseq	bed	narrowPeak	element quantifications	/data/reddylab/Revathy/collabs/Keith/ENCODE_submission/data/csaw_peaks/Th1.atacSTARR.pooled.3window_minTest.weighted.csaw.mm10.noATACpeaks.0bp_min.0.01.bed.gz	/awards/UM1HG009428/	/labs/tim-reddy/	tim-reddy:mmTh1-atac-starrseq-csaw-peakcall	mm10	tim-reddy:mmTh1-atac-starrseq-rep1-bam, tim-reddy:mmTh1-atac-starrseq-rep2-bam
tim-reddy:mmTh2-atac-starrseq	bed	narrowPeak	element quantifications	/data/reddylab/Revathy/collabs/Kei

In [7]:
!md5sum /data/reddylab/Revathy/collabs/Keith/ENCODE_submission/data/csaw_peaks/*.gz

c46632f24c5e1682f4bd714f8af12a6d  /data/reddylab/Revathy/collabs/Keith/ENCODE_submission/data/csaw_peaks/Th0.atacSTARR.pooled.3window_minTest.weighted.csaw.mm10.noATACpeaks.0bp_min.0.01.bed.gz
aae407944dd0b3999d8c48bb082e4fdb  /data/reddylab/Revathy/collabs/Keith/ENCODE_submission/data/csaw_peaks/Th17.atacSTARR.pooled.3window_minTest.weighted.csaw.mm10.noATACpeaks.0bp_min.0.01.bed.gz
446b18950da9cc65edd9baa2c2498858  /data/reddylab/Revathy/collabs/Keith/ENCODE_submission/data/csaw_peaks/Th1.atacSTARR.pooled.3window_minTest.weighted.csaw.mm10.noATACpeaks.0bp_min.0.01.bed.gz
046f9901092c956f6dd4606888d7953f  /data/reddylab/Revathy/collabs/Keith/ENCODE_submission/data/csaw_peaks/Th2.atacSTARR.pooled.3window_minTest.weighted.csaw.mm10.noATACpeaks.0bp_min.0.01.bed.gz
a50d9c97b95a84030647e8a2370b5de3  /data/reddylab/Revathy/collabs/Keith/ENCODE_submission/data/csaw_peaks/Treg.atacSTARR.pooled.3window_minTest.weighted.csaw.mm10.noATACpeaks.0bp_min.0.01.bed.gz


In [6]:
%%bash
source /data/reddylab/software/miniconda3/bin/activate revathy_py3
cd /data/reddylab/Revathy/collabs/Keith/ENCODE_submission/metadata
eu_register.py -m prod -p file -i mmCD4-PooledASTARRData-peak-calls.txt 

2023-04-24 12:50:54,360:eu_debug:	Connecting to www.encodeproject.org
2023-04-24 12:50:54,407:eu_debug:	submission=False: In non-submission mode.
2023-04-24 12:50:55,063:eu_debug:	submission=True: In submission mode.
2023-04-24 12:50:55,068:eu_debug:	
IN post().
2023-04-24 12:50:55,072:eu_debug:	<<<<<< POST file record tim-reddy:mmTh0-atac-starrseq-csaw-peakcall To DCC with URL https://www.encodeproject.org/file and this payload:

{
  "aliases": [
    "tim-reddy:mmTh0-atac-starrseq-csaw-peakcall"
  ],
  "assembly": "mm10",
  "award": "/awards/UM1HG009428/",
  "dataset": "tim-reddy:mmTh0-atac-starrseq",
  "derived_from": [
    "tim-reddy:mmTh0-atac-starrseq-rep1-bam",
    "tim-reddy:mmTh0-atac-starrseq-rep2-bam"
  ],
  "file_format": "bed",
  "file_format_type": "narrowPeak",
  "file_size": 396174,
  "lab": "/labs/tim-reddy/",
  "md5sum": "c46632f24c5e1682f4bd714f8af12a6d",
  "output_type": "element quantifications",
  "submitted_file_name": "/data/reddylab/Revathy/collabs/Keith/ENCODE_

Correct the CRADLE bed files to chnge the name of the chromosomes

In [23]:
%%bash
cell_types=(\ Th0 \
              Th1 \
              Th17 \
              Th2 \
              Treg)
for cell_type in ${cell_types[@]};
do 
gzip -c /data/reddylab/Revathy/collabs/Keith/ENCODE_submission/data/csaw_peaks/${cell_type}.atacSTARR.pooled.3window_minTest.weighted.csaw.mm10.noATACpeaks.0bp_min.0.01.corrected.bed \
> /data/reddylab/Revathy/collabs/Keith/ENCODE_submission/data/csaw_peaks/${cell_type}.atacSTARR.pooled.3window_minTest.weighted.csaw.mm10.noATACpeaks.0bp_min.0.01.corrected.bed.gz
done

In [25]:
!ls /data/reddylab/Alex/software/encValData/as/narrowPeak.as

/data/reddylab/Alex/software/encValData/as/narrowPeak.as


In [29]:
%%bash
cell_types=(\ Th0 \
              Th1 \
              Th17 \
              Th2 \
              Treg)
              
for cell_type in ${cell_types[@]};
do 
/data/common/shared_conda_envs/ucsc/bin/validateFiles \
-type=bed6+4 \
-chromInfo=/data/reddylab/software/encValData/mm10/chrom.sizes \
-as=/data/reddylab/Alex/software/encValData/as/narrowPeak.as \
/data/reddylab/Revathy/collabs/Keith/ENCODE_submission/data/csaw_peaks/${cell_type}.atacSTARR.pooled.3window_minTest.weighted.csaw.mm10.noATACpeaks.0bp_min.0.01.corrected.bed.gz
done

Error count 0
Error count 0
Error count 0
Error count 0
Error count 0


In [30]:
!cat /data/reddylab/Revathy/collabs/Keith/ENCODE_submission/metadata/mmCD4-PooledASTARRData-peak-calls-corrected.txt

file_id	file_path
ENCFF428NIS	/data/reddylab/Revathy/collabs/Keith/ENCODE_submission/data/csaw_peaks/Th0.atacSTARR.pooled.3window_minTest.weighted.csaw.mm10.noATACpeaks.0bp_min.0.01.corrected.bed.gz
ENCFF709PFP	/data/reddylab/Revathy/collabs/Keith/ENCODE_submission/data/csaw_peaks/Th1.atacSTARR.pooled.3window_minTest.weighted.csaw.mm10.noATACpeaks.0bp_min.0.01.corrected.bed.gz
ENCFF667UIB	/data/reddylab/Revathy/collabs/Keith/ENCODE_submission/data/csaw_peaks/Th2.atacSTARR.pooled.3window_minTest.weighted.csaw.mm10.noATACpeaks.0bp_min.0.01.corrected.bed.gz
ENCFF788NIT	/data/reddylab/Revathy/collabs/Keith/ENCODE_submission/data/csaw_peaks/Th17.atacSTARR.pooled.3window_minTest.weighted.csaw.mm10.noATACpeaks.0bp_min.0.01.corrected.bed.gz
ENCFF449JPD	/data/reddylab/Revathy/collabs/Keith/ENCODE_submission/data/csaw_peaks/Treg.atacSTARR.pooled.3window_minTest.weighted.csaw.mm10.noATACpeaks.0bp_min.0.01.corrected.bed.gz

In [31]:
import pandas as pd
import encode_utils as eu
from encode_utils.connection import Connection

conn = Connection("prod")

peaks = pd.read_csv('/data/reddylab/Revathy/collabs/Keith/ENCODE_submission/metadata/mmCD4-PooledASTARRData-peak-calls-corrected.txt', sep='\t')

for peak in range(len(peaks)):
    conn.upload_file(file_id=peaks['file_id'][peak], file_path=peaks['file_path'][peak])

2023-04-26 12:39:32,943:eu_debug:	Connecting to www.encodeproject.org
2023-04-26 12:39:32,968:eu_debug:	submission=False: In non-submission mode.
2023-04-26 12:39:33,694:eu_debug:	
IN upload_file()

2023-04-26 12:39:33,697:eu_debug:	Attempting to generate new file upload credentials
2023-04-26 12:39:34,511:eu_debug:	Success: upload credentials for 'ENCFF428NIS' regenerated.
2023-04-26 12:39:34,513:eu_debug:	>>>>>>GET ENCFF428NIS From DCC with URL https://www.encodeproject.org/ENCFF428NIS/?format=json
2023-04-26 12:39:35,216:eu_debug:	Running command 'aws s3 cp /data/reddylab/Revathy/collabs/Keith/ENCODE_submission/data/csaw_peaks/Th0.atacSTARR.pooled.3window_minTest.weighted.csaw.mm10.noATACpeaks.0bp_min.0.01.corrected.bed.gz s3://encode-files/2023/04/21/d2056bf4-eecb-4e2c-a6f3-2c7b9bbf3fd5/ENCFF428NIS.bed.gz'.
2023-04-26 12:39:43,201:eu_debug:	AWS upload successful.
2023-04-26 12:39:43,204:eu_debug:	
IN upload_file()

2023-04-26 12:39:43,205:eu_debug:	Attempting to generate new file u

In [32]:
!md5sum /data/reddylab/Revathy/collabs/Keith/ENCODE_submission/data/csaw_peaks/*.corrected.bed.gz

31d4194ac7bb49fe965787b5f114b66f  /data/reddylab/Revathy/collabs/Keith/ENCODE_submission/data/csaw_peaks/Th0.atacSTARR.pooled.3window_minTest.weighted.csaw.mm10.noATACpeaks.0bp_min.0.01.corrected.bed.gz
e9874375556b43742bbfe6163e7c79ec  /data/reddylab/Revathy/collabs/Keith/ENCODE_submission/data/csaw_peaks/Th17.atacSTARR.pooled.3window_minTest.weighted.csaw.mm10.noATACpeaks.0bp_min.0.01.corrected.bed.gz
4ee95ea38519f003de31677136d086e9  /data/reddylab/Revathy/collabs/Keith/ENCODE_submission/data/csaw_peaks/Th1.atacSTARR.pooled.3window_minTest.weighted.csaw.mm10.noATACpeaks.0bp_min.0.01.corrected.bed.gz
fa3ebbbd0a98f1b38af79e1852674733  /data/reddylab/Revathy/collabs/Keith/ENCODE_submission/data/csaw_peaks/Th2.atacSTARR.pooled.3window_minTest.weighted.csaw.mm10.noATACpeaks.0bp_min.0.01.corrected.bed.gz
95220f247d6859f1e8196b976b665dc1  /data/reddylab/Revathy/collabs/Keith/ENCODE_submission/data/csaw_peaks/Treg.atacSTARR.pooled.3window_minTest.weighted.csaw.mm10.noATACpeaks.0bp_min.0.01.